# National Health Insurance (NHI) – Predicting Healthcare Service Demand (South Africa)

**Technical Programming 2 – Assessment 4**  
**Submission Date:** 26 August 2025  

---

## Group & Admin Cell (Alphabetical by Surname)

- Ncobeni, N.A – 22412310  
- Nikwe, T – 22309434  
- Ngeleka, S.K – 22221194  
- Mathonsi, S.S – 22339617  
- Mazibuko, S.R – 22000786  
- Sigwaza, S.N – 22460979  
- Zungula, E – 22339408  

---

## Task Choice
We selected **TASK ONE: Predicting Healthcare Service Demand**.  

### Why this task?
- Aligns directly with NHI rollout needs: anticipating demand to allocate funding and capacity.  
- South African public microdata with health modules and demographics are publicly available (e.g., **GHS 2024**).  
- Natural, supervised-learning framing with clear targets (e.g., probability of clinic/hospital visit).  


## Dataset Sources (Credibility & Validity)

For this project we used **open, credible, multi-relational datasets** from official sources:

1. **General Household Survey 2024 (Stats SA)**  
   - Repository: World Bank Microdata Library (Released 8 July 2025)  
   - Link: [https://microdata.worldbank.org/index.php/catalog/6792](https://microdata.worldbank.org/index.php/catalog/6792)

2. **General Household Survey Overview (Stats SA)**  
   - Link: [https://www.statssa.gov.za/?page_id=16405](https://www.statssa.gov.za/?page_id=16405)

3. **South Africa Health Facility Master List (Nature Scientific Data)**  
   - Maina et al., 2019. “A spatial database of health facilities …”  
   - Link: [https://www.nature.com/articles/s41597-019-0142-2](https://www.nature.com/articles/s41597-019-0142-2)


In [ ]:
# Basic imports
import os, pathlib, numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

DATA_DIR = Path('data')
DATA_DIR.mkdir(exist_ok=True)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [ ]:
def load_or_sample():
    def file_if_exists(name):
        p = DATA_DIR / name
        return p if p.exists() else None

    hh_path = file_if_exists('ghs2024_households.csv')
    pr_path = file_if_exists('ghs2024_persons.csv')
    hv_path = file_if_exists('ghs2024_health_visits.csv')
    fc_path = file_if_exists('facilities_masterlist.csv')

    use_sample = not all([hh_path, pr_path, hv_path, fc_path])

    if use_sample:
        provinces = ['EC','FS','GP','KZN','LP','MP','NC','NW','WC']
        ownerships = ['Public','Private']

        facilities = pd.DataFrame({
            'facility_id': range(1,121),
            'province': np.random.choice(provinces, 120),
            'district': np.random.randint(1, 45, 120),
            'ownership': np.random.choice(ownerships, 120, p=[0.75,0.25]),
            'level': np.random.choice(['Primary','Secondary','Tertiary'], 120, p=[0.6,0.3,0.1]),
            'lat': -34 + np.random.rand(120)*10,
            'lon': 18 + np.random.rand(120)*8,
            'services_offered': np.random.choice(['General','Maternity','HIV/TB','Chronic'], 120)
        })

        households = pd.DataFrame({
            'hh_id': range(1,1001),
            'province': np.random.choice(provinces, 1000),
            'district': np.random.randint(1,45,1000),
            'urban_rural': np.random.choice(['Urban','Rural'], 1000, p=[0.65,0.35]),
            'income': np.random.lognormal(mean=9.5, sigma=0.6, size=1000).round(0),
            'household_size': np.random.randint(1,9,1000)
        })

        persons = pd.DataFrame({
            'person_id': range(1,3801),
            'hh_id': np.random.choice(households['hh_id'], 3800),
            'age': np.clip(np.random.normal(35, 18, 3800).round(0), 0, 95).astype(int),
            'sex': np.random.choice(['Male','Female'], 3800),
            'education': np.random.choice(['None','Primary','Secondary','Tertiary'], 3800, p=[0.05,0.25,0.5,0.2]),
            'employment': np.random.choice(['Employed','Unemployed','Inactive'], 3800, p=[0.45,0.35,0.2]),
            'medical_aid': np.random.choice([0,1], 3800, p=[0.75,0.25]),
            'chronic_condition': np.random.choice([0,1], 3800, p=[0.7,0.3])
        })

        hv = []
        for pid in persons['person_id']:
            n = np.random.poisson(1.7)
            for _ in range(n):
                fac = facilities.sample(1).iloc[0]
                hv.append({
                    'visit_id': len(hv)+1,
                    'person_id': pid,
                    'facility_id': int(fac['facility_id']),
                    'facility_type': np.random.choice(['Clinic','Hospital','CHC','GP'], p=[0.55,0.25,0.1,0.1]),
                    'reason': np.random.choice(['Acute','Chronic','Maternal','Injury','Other'], p=[0.35,0.25,0.1,0.1,0.2]),
                    'inpatient': np.random.choice([0,1], p=[0.9,0.1]),
                    'month': np.random.randint(1,13),
                    'distance_km': abs(np.random.normal(6, 4))
                })
        health_visits = pd.DataFrame(hv)
    else:
        households = pd.read_csv(hh_path)
        persons = pd.read_csv(pr_path)
        health_visits = pd.read_csv(hv_path)
        facilities = pd.read_csv(fc_path)

    return households, persons, health_visits, facilities, use_sample

households, persons, health_visits, facilities, using_sample = load_or_sample()

print("Using SAMPLE data" if using_sample else "Using REAL CSVs from data/")
households.head(), persons.head(), health_visits.head(), facilities.head()

In [ ]:
# Quick EDA
print("Rows: households=%d, persons=%d, visits=%d, facilities=%d" %
      (len(households), len(persons), len(health_visits), len(facilities)))

# Household urban/rural split
display(households['urban_rural'].value_counts().to_frame('count'))

# Age summary
display(persons['age'].describe())

In [ ]:
# Build features for demand prediction (whether a person had any visit)
visit_counts = health_visits.groupby('person_id').size().rename('visits_year').reset_index()
visit_counts['demand'] = (visit_counts['visits_year'] > 0).astype(int)
persons_aug = persons.merge(visit_counts[['person_id','demand']], on='person_id', how='left')
persons_aug['demand'] = persons_aug['demand'].fillna(0).astype(int)

# Facility density per district
fac_per_district = facilities.groupby(['province','district']).size().rename('fac_count').reset_index()

# Household and district population proxies
hh_pop = persons.groupby('hh_id').size().rename('hh_persons').reset_index()
households_aug = households.merge(hh_pop, on='hh_id', how='left').fillna({'hh_persons':0})
dist_pop = households_aug.groupby(['province','district'])['hh_persons'].sum().rename('district_pop').reset_index()
dist_fac = fac_per_district.merge(dist_pop, on=['province','district'], how='left')
dist_fac['fac_per_10k'] = (dist_fac['fac_count'] / dist_fac['district_pop'].replace(0, np.nan) * 10000).fillna(0)

# Join features to person
X = persons_aug.merge(households[['hh_id','province','district','urban_rural','income','household_size']],
                      on='hh_id', how='left')
X = X.merge(dist_fac[['province','district','fac_per_10k']], on=['province','district'], how='left')
X['fac_per_10k'] = X['fac_per_10k'].fillna(0)
y = X['demand']

X.head()

In [ ]:
numeric_features = ['age','income','household_size','fac_per_10k']
categorical_features = ['sex','education','employment','medical_aid','chronic_condition','urban_rural','province']

preprocess = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

clf = RandomForestClassifier(
    n_estimators=300,
    min_samples_split=4,
    min_samples_leaf=2,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

pipe = Pipeline(steps=[('prep', preprocess), ('model', clf)])

X_model = X[numeric_features + categorical_features]
X_train, X_test, y_train, y_test = train_test_split(X_model, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
y_prob = pipe.predict_proba(X_test)[:,1]

metrics = {
    'ROC-AUC': roc_auc_score(y_test, y_prob),
    'Accuracy': accuracy_score(y_test, y_pred),
    'Precision': precision_score(y_test, y_pred),
    'Recall': recall_score(y_test, y_pred),
    'F1': f1_score(y_test, y_pred)
}
metrics

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)
print("\nClassification report:\n", classification_report(y_test, y_pred))

In [ ]:
rf = pipe.named_steps['model']
ohe = pipe.named_steps['prep'].named_transformers_['cat']
cat_names = list(ohe.get_feature_names_out(categorical_features))
all_names = numeric_features + cat_names

importances = pd.Series(rf.feature_importances_, index=all_names).sort_values(ascending=False)
importances.head(20)

In [ ]:
# Plot top 15 feature importances
topk = importances.head(15).sort_values()
plt.figure(figsize=(8,6))
topk.plot(kind='barh')
plt.title('Top Feature Importances')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

# Province-level mean probability
all_prob = pipe.predict_proba(X_model)[:,1]
scored = X.copy()
scored['prob'] = all_prob
plt.figure(figsize=(8,4))
scored.groupby('province')['prob'].mean().sort_values().plot(kind='bar')
plt.title('Predicted Demand Probability by Province')
plt.ylabel('Mean probability')
plt.tight_layout()
plt.show()

## Conclusions
- The pipeline predicts whether an individual will use health services within the year.  
- Feature importance suggests **socio-demographics** and **local facility density** are strong drivers of demand.  
- Policy angle: districts with low **facilities per 10k residents** and high predicted demand could be prioritised for capacity or outreach.  

> Reproducibility: place real CSVs in `data/` as `ghs2024_households.csv`, `ghs2024_persons.csv`, `ghs2024_health_visits.csv`, `facilities_masterlist.csv`. If absent, this notebook uses synthetic sample data so it still runs end-to-end.